# 02 - Preprocesado de Datos  
En este notebook se realiza el preprocesado del dataset de fútbol internacional entre 1872 y 2012, incluyendo:

1. Tratamiento de valores nulos  
2. Eliminación y análisis de duplicados  
3. Normalización de tipos  
4. Creación de columnas adicionales  
5. Preparación para modelos secuenciales

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

Se carga el .zip del dataset y se descomprime

In [2]:
uploaded = files.upload()
!unzip archive.zip

Saving archive.zip to archive.zip
Archive:  archive.zip
  inflating: former_names.csv        
  inflating: goalscorers.csv         
  inflating: results.csv             
  inflating: shootouts.csv           


Se cargan los archivos del Dataset

In [3]:
df_results = pd.read_csv("results.csv")
df_goalscorers = pd.read_csv("goalscorers.csv")
df_former = pd.read_csv("former_names.csv")
df_shootouts = pd.read_csv("shootouts.csv")

Conversión de tipos de datos
Especialmente convertir la fecha a datetime

In [4]:
df_results['date'] = pd.to_datetime(df_results['date'], errors = 'coerce')

#Otras columnas numericas
df_results['home_score'] = pd.to_numeric(df_results['home_score'], errors='coerce')
df_results['away_score'] = pd.to_numeric(df_results['away_score'], errors='coerce')

Tratamiento de valores nulos

In [5]:
print("Nulos por dataset:")
print(df_results.isnull().sum(), "\n")
print(df_goalscorers.isnull().sum(), "\n")
print(df_former.isnull().sum(), "\n")
print(df_shootouts.isnull().sum())

Nulos por dataset:
date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64 

date           0
home_team      0
away_team      0
team           0
scorer        48
minute       256
own_goal       0
penalty        0
dtype: int64 

current       0
former        0
start_date    0
end_date      0
dtype: int64 

date               0
home_team          0
away_team          0
winner             0
first_shooter    417
dtype: int64


Limpieza Básica de nulos

In [6]:
# En results, no queremos perder información de fechas
df_results = df_results.dropna(subset=['date'])

# Rellenar campos de estadio cuando estén vacíos
df_results['city'] = df_results['city'].fillna('Unknown')
df_results['country'] = df_results['country'].fillna('Unknown')

# En goalscorers puede haber datos faltantes irrelevantes
df_goalscorers = df_goalscorers.dropna(subset=['scorer'])

# former_names — rellenar faltantes si existen (nuevo método)
df_former = df_former.ffill()

# shootouts — valores faltantes no afectan las columnas críticas
df_shootouts = df_shootouts.fillna("Unknown")

Eliminacion de duplicados

In [7]:
df_results = df_results.drop_duplicates()
df_goalscorers = df_goalscorers.drop_duplicates()
df_former = df_former.drop_duplicates()
df_shootouts = df_shootouts.drop_duplicates()

Creación de nuevas columnas

Año, Diferencia de goles, ganador, etc

In [8]:
# Año del partido
df_results['year'] = df_results['date'].dt.year

# Diferencia de goles
df_results['goal_diff'] = df_results['home_score'] - df_results['away_score']

# Determinar ganador
def get_winner(row):
    if row.home_score > row.away_score:
        return row.home_team
    elif row.home_score < row.away_score:
        return row.away_team
    else:
        return "Draw"

df_results['winner'] = df_results.apply(get_winner, axis=1)

Unificación parcial de datasets

In [9]:
df_goalscorers['date'] = pd.to_datetime(df_goalscorers['date'], errors='coerce')
df_results['date'] = pd.to_datetime(df_results['date'], errors='coerce')

df_merged = df_goalscorers.merge(
    df_results,
    left_on=['date', 'team'],
    right_on=['date', 'home_team'],
    how='left'
)

df_merged.head()

,date,home_team_x,away_team_x,team,scorer,minute,own_goal,penalty,home_team_y,away_team_y,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False,Argentina,Chile,6.0,1.0,Copa América,Buenos Aires,Argentina,False,1916.0,5.0,Argentina


Normalizacion de nombres de países

Esto evita problemas como USA vs United States

In [10]:
df_results['home_team'] = df_results['home_team'].str.title()
df_results['away_team'] = df_results['away_team'].str.title()
df_former['current'] = df_former['current'].str.title()

Guardar Datasets Procesados

Estos archivos se van a usar en el notebook 3

In [11]:
df_results.to_csv("results_clean.csv", index=False)
df_goalscorers.to_csv("goalscorers_clean.csv", index=False)
df_former.to_csv("former_names_clean.csv", index=False)
df_shootouts.to_csv("shootouts_clean.csv", index=False)

Descarga de los archivos generados

In [12]:
import shutil
import zipfile

zip_filename = "datasets_clean.zip"

with zipfile.ZipFile(zip_filename, 'w') as z:
    z.write("results_clean.csv")
    z.write("goalscorers_clean.csv")
    z.write("former_names_clean.csv")
    z.write("shootouts_clean.csv")

from google.colab import files
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>